In [1]:
import pandas as pd
from sqlalchemy import create_engine
from string import digits
import numpy as np
import os
import nltk
import string
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI

In [2]:
# from nltk.corpus import brown
# brown.words()

In [7]:
csv1 = "./indeed_job_dataset_jds.csv"
# csv2 = "./indeed_job_dataset_V5_CSV.csv"
df1 = pd.read_csv(csv1)
# df2 = pd.read_csv(csv2)
df1.head()
# df2.head()

,Index_No,Job_Title,Link,Queried_Salary,Job_Type,Skill,Description
0,0,Data Scientist,https://www.indeed.com/rc/clk?jk=6a105f495c36a...,<80000,data_scientist,"['SAP', 'SQL']","[<p><b>POSITION SUMMARY</b></p>, <p>\r\r\nThe ..."
1,1,Data Scientist,https://www.indeed.com/rc/clk?jk=86afd561ea8c6...,<80000,data_scientist,"['Machine Learning', 'R', 'SAS', 'SQL', 'Python']","[<p><b>What do we need?</b></p>, <ul><li>\r\r\..."
2,2,Data Scientist,https://www.indeed.com/rc/clk?jk=e0aad317e6d45...,<80000,data_scientist,"['Data Mining', 'Data Management', 'R', 'SAS',...","[<ul><li>Validate, analyze, and conduct statis..."
3,3,Graduate Studies Program - Data Scientist,https://www.indeed.com/rc/clk?jk=1cfdd9e391a63...,<80000,data_scientist,['Certified Internal Auditor'],"[<p>Full time</p>, <p>Washington, DC metro are..."
4,4,Data Scientist I,https://www.indeed.com/rc/clk?jk=fec647775a21e...,<80000,data_scientist,"['Statistical Software', 'Time Management', 'R...",[<ul><li>Assist in consultations with business...


In [9]:
df = df1[["Job_Type", "Description"]]

In [4]:
# # Combine the data into a single dataset.  
# df = pd.merge(df1, df2, how="left", on=["Index_No"])
# df.head()

,Index_No,Description,Job_Title,Job_Type,Link,Queried_Salary,<80000,80000-99999,100000-119999,120000-139999,...,spss,softwaredevelopment,shellscripting,datascience,docker,mongodb,.net,projectmanagement,businessintelligence,s3
0,0,"[<p><b>POSITION SUMMARY</b></p>, <p>\r\r\nThe ...",Data Scientist,Data Scientist,https://www.indeed.com/rc/clk?jk=6a105f495c36a...,<80000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,"[<p><b>What do we need?</b></p>, <ul><li>\r\r\...",Data Scientist,Data Scientist,https://www.indeed.com/rc/clk?jk=86afd561ea8c6...,<80000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,"[<ul><li>Validate, analyze, and conduct statis...",Data Scientist,Data Scientist,https://www.indeed.com/rc/clk?jk=e0aad317e6d45...,<80000,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,3,"[<p>Full time</p>, <p>Washington, DC metro are...",Graduate Studies Program - Data Scientist,Data Scientist,https://www.indeed.com/rc/clk?jk=1cfdd9e391a63...,<80000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,[<ul><li>Assist in consultations with business...,Data Scientist I,Data Scientist,https://www.indeed.com/rc/clk?jk=fec647775a21e...,<80000,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [5]:
# df3 = df[['Job_Type', 'Description']]
# df3.head()

,Job_Type,Description
0,Data Scientist,"[<p><b>POSITION SUMMARY</b></p>, <p>\r\r\nThe ..."
1,Data Scientist,"[<p><b>What do we need?</b></p>, <ul><li>\r\r\..."
2,Data Scientist,"[<ul><li>Validate, analyze, and conduct statis..."
3,Data Scientist,"[<p>Full time</p>, <p>Washington, DC metro are..."
4,Data Scientist,[<ul><li>Assist in consultations with business...


In [ ]:
# df.to_csv('df.csv')
# df.describe()
# df.dtypes
# len(df3['Description'])

In [11]:
# df3=df3['Description'].replace('', nan_value, inplace=True)
# df3=df3.dropna(subset=['Description'], inplace=True)

df1['Description'].dropna(inplace=True)
len(df1['Description'])

5413

In [20]:
data = [char for char in df1['Description'] if char not in string.punctuation]
print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [23]:
data_1=''.join(data)
data_1.split()

['[<p><b>POSITION',
 'SUMMARY</b></p>,',
 '<p>',
 'The',
 'Business',
 'Analyst',
 'role',
 'is',
 'the',
 'primary',
 'architect',
 'of',
 'reporting',
 'and',
 'dashboard',
 'solutions',
 'for',
 'internal',
 'and',
 'external',
 'clients.',
 'Utilizing',
 'ESI',
 'corporate',
 'standard',
 'development',
 'tools',
 'this',
 'position',
 'is',
 'responsible',
 'for',
 'the',
 'design,',
 'development,',
 'implementation,',
 'analysis,',
 'interpretation',
 'and',
 'communication',
 'of',
 'business',
 'information',
 'based',
 'on',
 'the',
 'needs',
 'of',
 'individual',
 'clients.',
 'The',
 'ability',
 'to',
 'balance',
 'overall',
 'aesthetics',
 'with',
 'robust',
 'and',
 'intuitive',
 'functionality',
 'is',
 'a',
 'critical',
 'requirement',
 'for',
 'success',
 'in',
 'this',
 'position.</p>,',
 '<p><b>',
 'ESSENTIAL',
 'FUNCTIONS</b></p>,',
 '<ul><li>',
 'Successfully',
 'design',
 'and',
 'implement',
 'external',
 'client',
 'data',
 'reporting',
 'and',
 'dashboard',
 's

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(df1["Description"])
print(vectorizer.vocabulary)

None


In [26]:
stopwords1 = nltk.corpus.stopwords.words('english')
new_words=('<p>','SUMMARY</b></p>','is,''</p>', '<b>', '</b>','<li>','</li>', '<ul>', '</ul>','p><b>POSITION',)
for i in new_words:
    stopwords1.append(i)
print(stopwords1)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [19]:
def process_text(text):
             clean_words = [word for word in text if word.lower() not in stopwords1]
             nopunc= [char for char in clean_words if char not in string.punctuation]
             nopunc=''.join(nopunc)
             return nopunc

In [15]:
def process_text(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords1]
    return clean_words.extend(new_words)

df1['Description'].head().apply(process_text)   

0    None
1    None
2    None
3    None
4    None
Name: Description, dtype: object

In [ ]:
example_sent = """This is a sample sentence, 
                  showing off the stop words filtration."""

jds = df3['Description']  

stop_words = set(stopwords.words('english'))  
  


In [ ]:
word_tokens = word_tokenize(example_sent)
# word_tokens = word_tokenize(jds)

In [ ]:
filtered_sentence = [w for w in word_tokens if not w in stop_words]  
filtered_sentence = []  
  
for w in word_tokens:  
    if w not in stop_words:  
        filtered_sentence.append(w)  
  
print(word_tokens)  
print(filtered_sentence)

In [ ]:
filtered_sentence1 = [w for w in jds if not w in stop_words]  
filtered_sentence1 = [] 

for w in jds:  
    if w not in stop_words:  
        filtered_sentence1.append(w)  
  
print(jds)  
print(filtered_sentence1)

In [ ]:
# class NaiveBayesClassifier(ClassifierI):